<a href="https://colab.research.google.com/github/DionKimmm/2019_Summer_DL_Prof_Seok/blob/master/190717_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN을 이용한 MNIST Data Set 식별

## 순서
* 필요한 라이브러리 불러 오기
* 데이터 불러오기
* CNN 신경망 모델 구성
* 최적화
* 훈련데이터를 이용한 학습
* 테스트 데이터를 통한 최종 식별 결과 확인

### 필요한 라이브러리 불러오기

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

### 학습, 테스트, 레이블 데이터 불러오기

In [2]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

W0717 06:25:05.174579 139687511222144 deprecation.py:323] From <ipython-input-2-6fa84048fdd1>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0717 06:25:05.180488 139687511222144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0717 06:25:05.181842 139687511222144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0717 06:25:05.427190 139687511222144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0717 06:25:05.429751 139687511222144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0717 06:25:05.479436 139687511222144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


### CNN 신경망 모델 구성(1)

In [0]:
# 데이터 정의
X = tf.placeholder(tf.float32, [None, 28, 28, 1]) # None : 입력 데이터의 개수는 정해져 있지 않음
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

### CNN 신경망 모델 구성(2) : 첫 번째 층의 구성

In [0]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### CNN 신경망 모델 구성(3) : 두 번째 층의 구성

In [0]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
### CNN 신경망 모델 구성(4) : 세 번째 층의 구성 : Fully Connected Layer

In [7]:
# L2의 모양이 뭐지?
print(L2)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [8]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob) # dropout

W0717 06:25:05.778964 139687511222144 deprecation.py:506] From <ipython-input-8-f10978440ea1>:5: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

### 최적화

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

W0717 06:25:05.821793 139687511222144 deprecation.py:323] From <ipython-input-10-b021224dac13>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 훈련 데이터를 신경망 모델 학습

In [11]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
  total_cost = 0
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape(-1, 28, 28, 1)
    _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
    total_cost += cost_val
  
  print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost = ', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')

Epoch: 0001 Avg. cost =  0.339
Epoch: 0002 Avg. cost =  0.107
Epoch: 0003 Avg. cost =  0.076
Epoch: 0004 Avg. cost =  0.061
Epoch: 0005 Avg. cost =  0.051
Epoch: 0006 Avg. cost =  0.042
Epoch: 0007 Avg. cost =  0.036
Epoch: 0008 Avg. cost =  0.032
Epoch: 0009 Avg. cost =  0.030
Epoch: 0010 Avg. cost =  0.026
Epoch: 0011 Avg. cost =  0.021
Epoch: 0012 Avg. cost =  0.019
Epoch: 0013 Avg. cost =  0.020
Epoch: 0014 Avg. cost =  0.017
Epoch: 0015 Avg. cost =  0.016
최적화 완료!


### 테스트 데이터를 이용한 최종 식별 결과 확인

In [12]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels, keep_prob: 1}))

정확도: 0.9914
